In [ ]:
# Clone code
%cd /content
!git clone https://github.com/comfyanonymous/ComfyUI.git ./ComfyUI
!git clone https://github.com/city96/ComfyUI-GGUF.git ./ComfyUI/custom_nodes/quantized

# Download library
%cd /content/ComfyUI
!pip install -q einops==0.8.0 torchsde==0.2.6 diffusers==0.31.0 accelerate==1.1.0 gguf==0.10.0 gradio==5.8.0
!pip install -q huggingface_hub==0.27.0 hf_transfer==0.1.8

# Download model
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download YarvixPA/FLUX.1-Fill-dev-gguf flux1-fill-dev-Q4_K_S.gguf --local-dir ./models/unet --local-dir-use-symlinks False
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download camenduru/FLUX.1-dev ae.sft --local-dir ./models/vae --local-dir-use-symlinks False
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download zer0int/CLIP-GmP-ViT-L-14 ViT-L-14-TEXT-detail-improved-hiT-GmP-TE-only-HF.safetensors --local-dir ./models/clip --local-dir-use-symlinks False
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download comfyanonymous/flux_text_encoders t5xxl_fp8_e4m3fn.safetensors --local-dir ./models/clip --local-dir-use-symlinks False

In [ ]:
# Set nodes
%cd /content/ComfyUI

import torch
import nodes
from nodes import NODE_CLASS_MAPPINGS
from comfy import model_management
from comfy_extras import nodes_custom_sampler, nodes_differential_diffusion, nodes_flux, nodes_model_advanced

CLIPVisionLoader = nodes.NODE_CLASS_MAPPINGS["CLIPVisionLoader"]()
LoadImage = nodes.NODE_CLASS_MAPPINGS["LoadImage"]()
InpaintModelConditioning = nodes.NODE_CLASS_MAPPINGS["InpaintModelConditioning"]()
DifferentialDiffusion = nodes_differential_diffusion.NODE_CLASS_MAPPINGS["DifferentialDiffusion"]()

DualCLIPLoader = nodes.NODE_CLASS_MAPPINGS["DualCLIPLoader"]()
VAELoader = nodes.NODE_CLASS_MAPPINGS["VAELoader"]()
CLIPTextEncode = nodes.NODE_CLASS_MAPPINGS["CLIPTextEncode"]()
FluxGuidance = nodes_flux.NODE_CLASS_MAPPINGS["FluxGuidance"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSampler = nodes.NODE_CLASS_MAPPINGS["KSampler"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()

RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
EmptyLatentImage = nodes.NODE_CLASS_MAPPINGS["EmptyLatentImage"]()
VAEDecode = nodes.NODE_CLASS_MAPPINGS["VAEDecode"]()

# Import model
with torch.inference_mode():
  clip = DualCLIPLoader.load_clip("t5xxl_fp8_e4m3fn.safetensors", "ViT-L-14-TEXT-detail-improved-hiT-GmP-TE-only-HF.safetensors", "flux")[0]
  vae = VAELoader.load_vae("ae.sft")[0]

from custom_nodes.quantized.nodes import NODE_CLASS_MAPPINGS

UNETLoader = NODE_CLASS_MAPPINGS["UnetLoaderGGUF"]()

with torch.inference_mode():
  unet = UNETLoader.load_unet("flux1-fill-dev-Q4_K_S.gguf")[0]

In [ ]:
import os
import random
import torch
import numpy as np
from PIL import Image, ImageOps
import gradio as gr

def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2

def resize_max(image):
    max_length = 1024
    width, height = image.size
    scaling_factor = min(max_length / width, max_length / height)

    if scaling_factor < 1:
        new_width = int(width * scaling_factor)
        new_height = int(height * scaling_factor)
        image = image.resize((new_width, new_height), Image.LANCZOS)

    return image

# Function to generate image
def generate_image(image, prompt, seed):
  with torch.inference_mode():
    positive_prompt = prompt
    seed = seed
    steps = 20
    sampler_name = "euler"
    scheduler = "simple"

    image_path = image['background']
    mask_path = image['layers'][0]

    image = Image.open(image_path).convert("RGBA")
    image.save("/content/ComfyUI/input/original.png")
    mask = Image.open(mask_path).convert("L")
    mask = ImageOps.invert(mask)
    mask.save("/content/ComfyUI/input/mask.png")

    width, height = image.size
    image_f = resize_max(image)
    mask_f = resize_max(mask)
    image_f.putalpha(mask_f)
    image_f.save("/content/ComfyUI/input/image.png")

    image_ff, mask_ff = LoadImage.load_image("image.png") # "image.png"

    cond = CLIPTextEncode.encode(text=prompt, clip=clip)[0]
    cond_f = FluxGuidance.append(conditioning=cond, guidance=30)[0]
    con_f_neg = CLIPTextEncode.encode(text="", clip=clip)[0]
    cond_inpaint, cond_neg_inpaint, latent = InpaintModelConditioning.encode(positive=cond_f, negative=con_f_neg, vae=vae, pixels=image_ff, mask=mask_ff, noise_mask=False)

    noise = RandomNoise.get_noise(seed)[0]
    new_unet = DifferentialDiffusion.apply(model=unet)[0]
    sample = KSampler.sample(model=new_unet, positive=cond_inpaint, negative=cond_neg_inpaint, latent_image=latent, seed=seed, steps=steps, cfg=1.0, sampler_name=sampler_name, scheduler=scheduler, denoise=1.0)[0]
    model_management.soft_empty_cache()

    decoded = VAEDecode.decode(vae=vae, samples=sample)[0].detach()
    image = Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0])

    return image

# Function to handle button click
def on_generate_click(image, prompt, seed):
    if not prompt.strip():
        return None, "❌ 텍스트를 입력해주세요.", None
    if not image:
        return None, "❌ 이미지를 입력해주세요.", None
    try:
        image = generate_image(image, prompt, seed)
        return image, "✅ 성공적으로 이미지를 생성했습니다.", seed
    except Exception as e:
        return None, f"❌ 에러 발생: {str(e)}", None

# Function to generate a random seed
def roll_seed():
    return random.randint(0, 2**32 - 1)

In [ ]:
# Create Gradio interface
def create_interface():
    with gr.Blocks() as demo:
        gr.Markdown("# 🖼️ 플럭스 활용하기")
        with gr.Row():
            # Left column: Inputs
            with gr.Column():
                image = gr.ImageEditor(
                    label="🖼️ 이미지",
                    type='filepath',
                    sources="upload",
                    brush=gr.Brush(colors=["#FFFFFF"], default_color="#FFFFFF", color_mode="fixed"),
                    height=1000,
                )
                prompt = gr.Textbox(
                    label="📝 텍스트",
                    placeholder="원하는 장면을 영어로 입력해주세요...",
                    lines=3
                )
                with gr.Row():
                    seed = gr.Number(
                        label="🎲 시드",
                        value=2024,
                        precision=0,
                        interactive=True
                    )
                    roll_seed_btn = gr.Button("🎲 랜덤 시드 만들기")
                generate_btn = gr.Button("🎨 이미지 생성하기")
                progress = gr.Textbox(label="🔄 현재 상태", interactive=False)

            # Right column: Output
            with gr.Column(scale=1):
                output = gr.Image(label="🖼️ 생성한 이미지")
                used_seed = gr.Number(label="📌 사용된 시드", value=42, precision=0, interactive=False)

        # Define the button click events
        generate_btn.click(
            fn=on_generate_click,
            inputs=[image, prompt, seed],
            outputs=[output, progress, used_seed]
        )

        roll_seed_btn.click(
            fn=roll_seed,
            inputs=None,
            outputs=seed
        )

    return demo

if __name__ == "__main__":
    interface = create_interface()
    interface.launch()